In [1]:
from pathlib import Path
import datetime as dt
import pandas as pd

In [2]:
# Read the data for EV registrations in California
df = pd.read_csv(
    Path("../../../../../data/raw_data/population_2010-2019.csv"),
    encoding="ISO-8859-1",
)

print("Shape:", df.shape)
display(df.head())

Shape: (3193, 164)


,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
0,40,3,6,1,0,Alabama,Alabama,4779736,4780125,4785437,...,1.917501,0.578434,1.186314,1.522549,0.563489,0.626357,0.745172,1.090366,1.773786,2.483744
1,50,3,6,1,1,Alabama,Autauga County,54571,54597,54773,...,4.847310,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183112,...,24.017829,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567
3,50,3,6,1,5,Alabama,Barbour County,27457,27455,27327,...,-5.690302,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664
4,50,3,6,1,7,Alabama,Bibb County,22915,22915,22870,...,1.385134,-4.998356,-3.787545,-5.797999,1.331144,1.329817,-0.708717,-3.234669,-6.857092,1.831952


In [3]:
for index, value in enumerate(df.columns):
    print(index, value)

0 SUMLEV
1 REGION
2 DIVISION
3 STATE
4 COUNTY
5 STNAME
6 CTYNAME
7 CENSUS2010POP
8 ESTIMATESBASE2010
9 POPESTIMATE2010
10 POPESTIMATE2011
11 POPESTIMATE2012
12 POPESTIMATE2013
13 POPESTIMATE2014
14 POPESTIMATE2015
15 POPESTIMATE2016
16 POPESTIMATE2017
17 POPESTIMATE2018
18 POPESTIMATE2019
19 NPOPCHG_2010
20 NPOPCHG_2011
21 NPOPCHG_2012
22 NPOPCHG_2013
23 NPOPCHG_2014
24 NPOPCHG_2015
25 NPOPCHG_2016
26 NPOPCHG_2017
27 NPOPCHG_2018
28 NPOPCHG_2019
29 BIRTHS2010
30 BIRTHS2011
31 BIRTHS2012
32 BIRTHS2013
33 BIRTHS2014
34 BIRTHS2015
35 BIRTHS2016
36 BIRTHS2017
37 BIRTHS2018
38 BIRTHS2019
39 DEATHS2010
40 DEATHS2011
41 DEATHS2012
42 DEATHS2013
43 DEATHS2014
44 DEATHS2015
45 DEATHS2016
46 DEATHS2017
47 DEATHS2018
48 DEATHS2019
49 NATURALINC2010
50 NATURALINC2011
51 NATURALINC2012
52 NATURALINC2013
53 NATURALINC2014
54 NATURALINC2015
55 NATURALINC2016
56 NATURALINC2017
57 NATURALINC2018
58 NATURALINC2019
59 INTERNATIONALMIG2010
60 INTERNATIONALMIG2011
61 INTERNATIONALMIG2012
62 INTERNATIONALMI

In [6]:
selected_columns = ["STATE", "COUNTY", "STNAME", "CTYNAME"] + \
    [f"POPESTIMATE{year}" for year in range(2010, 2019 + 1)]

df = df.loc[df["STNAME"] == "Texas", selected_columns]

print("Shape:", df.shape)
display(df.head())

Shape: (255, 14)


,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019
2566,48,0,Texas,Texas,25241971,25645629,26084481,26480266,26964333,27470056,27914410,28295273,28628666,28995881
2567,48,1,Texas,Anderson County,58493,58394,58059,57960,57829,57639,57526,58175,57974,57735
2568,48,3,Texas,Andrews County,14849,15386,16105,16779,17435,18072,17791,17603,18011,18705
2569,48,5,Texas,Angelina County,86905,87297,87509,87312,87567,87829,87682,87572,86810,86715
2570,48,7,Texas,Aransas County,23181,23216,23459,23889,24566,24800,25162,25392,23446,23510
